In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from glob import glob
import os



In [2]:
DATA_DIR = '/home/chicm/ml/kgdata/species'
TRAIN_DIR = DATA_DIR + '/train'
RESULT_DIR = DATA_DIR + '/results'

TRAIN_FEAT = RESULT_DIR + '/train_feats.dat'
VAL_FEAT = RESULT_DIR + '/val_feats.dat'


#VAL_DIR = DATA_DIR + '/val-224'

batch_size = 64

In [3]:
df_train = pd.read_csv(DATA_DIR+'/train_labels.csv')
master = df_train
master.head()

,name,invasive
0,1,0
1,2,0
2,3,1
3,4,0
4,5,1


In [4]:
img_path = TRAIN_DIR+'/'

In [5]:
y = []
file_paths = []
for i in range(len(master)):
    file_paths.append( TRAIN_DIR + '/' + str(master.ix[i][0]) +'.jpg' )
    y.append(master.ix[i][1])
y = np.array(y)
print(y.shape)

(2295,)


In [6]:
print(file_paths[:5])

['/home/chicm/ml/kgdata/species/train/1.jpg', '/home/chicm/ml/kgdata/species/train/2.jpg', '/home/chicm/ml/kgdata/species/train/3.jpg', '/home/chicm/ml/kgdata/species/train/4.jpg', '/home/chicm/ml/kgdata/species/train/5.jpg']


In [7]:
#image reseize & centering & crop 

def centering_image(img):
    size = [256,256]
    
    img_size = img.shape[:2]
    
    # centering
    row = (size[1] - img_size[0]) // 2
    col = (size[0] - img_size[1]) // 2
    resized = np.zeros(list(size) + [img.shape[2]], dtype=np.uint8)
    resized[row:(row + img.shape[0]), col:(col + img.shape[1])] = img

    return resized


x = []
for i, file_path in enumerate(file_paths):
    #read image
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #resize
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = centering_image(cv2.resize(img, dsize=tile_size))
    
    #out put 224*224px 
    img = img[16:240, 16:240]
    x.append(img)

x = np.array(x)
print(x.shape)


(2295, 224, 224, 3)


In [8]:
sample_submission = pd.read_csv(DATA_DIR+"/sample_submission.csv")
img_path = DATA_DIR+"/test/"

test_names = []
file_paths = []

for i in range(len(sample_submission)):
    test_names.append(sample_submission.ix[i][0])
    file_paths.append( img_path + str(int(sample_submission.ix[i][0])) +'.jpg' )
    
test_names = np.array(test_names)
print(test_names.shape)

(1531,)


In [9]:
test_images = []
for file_path in file_paths:
    #read image
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #resize
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    #centering
    img = centering_image(cv2.resize(img, dsize=tile_size))
    
    #out put 224*224px 
    img = img[16:240, 16:240]
    test_images.append(img)
    
    path, ext = os.path.splitext( os.path.basename(file_paths[0]) )

test_images = np.array(test_images)

## split train and val

In [10]:
data_num = len(y)
random_index = np.random.permutation(data_num)

x_shuffle = []
y_shuffle = []
for i in range(data_num):
    x_shuffle.append(x[random_index[i]])
    y_shuffle.append(y[random_index[i]])
    
x = np.array(x_shuffle) 
y = np.array(y_shuffle)

In [11]:
val_split_num = int(round(0.2*len(y)))
x_train = x[val_split_num:]
y_train = y[val_split_num:]
x_test = x[:val_split_num]
y_test = y[:val_split_num]

print('x_train', x_train.shape)
print('y_train', y_train.shape)
print('x_test', x_test.shape)
print('y_test', y_test.shape)

x_train (1836, 224, 224, 3)
y_train (1836,)
x_test (459, 224, 224, 3)
y_test (459,)


In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Use VGG

In [13]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.optimizers import Adam
from keras.layers import Dropout, Flatten, Dense

img_rows, img_cols, img_channel = 224, 224, 3

Using TensorFlow backend.


In [14]:
incep_model = applications.InceptionV3(include_top=False, 
                                       weights='imagenet', 
                                       input_shape=(img_rows, img_cols, img_channel))

In [15]:
vgg16_model = applications.VGG16(weights='imagenet', 
                                 include_top=False, 
                                 input_shape=(img_rows, img_cols, img_channel))

In [16]:
vgg19_model = applications.VGG19(include_top=False, 
                                 weights='imagenet', 
                                 input_shape=(img_rows, img_cols, img_channel))

In [17]:
res_model = applications.ResNet50(
            include_top=False, 
            weights='imagenet',
            input_tensor=None, 
            input_shape=(img_rows, img_cols, img_channel),
            pooling=None)

In [19]:
#base_model.output_shape[1:]
def get_dense_model(input_shape):
    dense_model = Sequential()
    dense_model.add(Flatten(input_shape=input_shape))
    dense_model.add(Dense(256, activation='relu'))
    dense_model.add(Dropout(0.5))
    dense_model.add(Dense(1, activation='sigmoid'))
    return dense_model

In [21]:
models = []
models.append(Model(inputs=vgg16_model.input, 
                    outputs=get_dense_model(vgg16_model.output_shape[1:])(vgg16_model.output)))
models.append(Model(inputs=vgg19_model.input, 
                    outputs=get_dense_model(vgg19_model.output_shape[1:])(vgg19_model.output)))
models.append(Model(inputs=incep_model.input, 
                    outputs=get_dense_model(incep_model.output_shape[1:])(incep_model.output)))

for model in models:
    model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

#model.summary()

In [22]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint


w_file = RESULT_DIR + '/w_conv_{}.h5'

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)

In [23]:
from keras.callbacks import LearningRateScheduler
def lr_schedule(epoch):
    if epoch <= 50:
        return 0.0001
    else: 
        return 0.00005

In [24]:
batch_size = 32
epochs = 100

for i, model in enumerate(models):
    history = model.fit_generator(
        train_datagen.flow(x_train, y_train, batch_size=batch_size),
        steps_per_epoch=x_train.shape[0] // batch_size,
        epochs=epochs,
        validation_data=(x_test, y_test), verbose=2,
        callbacks=[ModelCheckpoint(w_file.format(i), 
                                   monitor='val_acc', 
                                   save_best_only=True),
                                   LearningRateScheduler(lr_schedule)])

Epoch 1/100
28s - loss: 0.5880 - acc: 0.6908 - val_loss: 0.4165 - val_acc: 0.7974
Epoch 2/100
26s - loss: 0.3559 - acc: 0.8333 - val_loss: 0.2543 - val_acc: 0.8998
Epoch 3/100
25s - loss: 0.2773 - acc: 0.8796 - val_loss: 0.1968 - val_acc: 0.9237
Epoch 4/100
25s - loss: 0.2118 - acc: 0.9128 - val_loss: 0.1734 - val_acc: 0.9281
Epoch 5/100
25s - loss: 0.2005 - acc: 0.9174 - val_loss: 0.1473 - val_acc: 0.9499
Epoch 6/100
25s - loss: 0.1682 - acc: 0.9426 - val_loss: 0.1865 - val_acc: 0.9194
Epoch 7/100
25s - loss: 0.1513 - acc: 0.9476 - val_loss: 0.1233 - val_acc: 0.9521
Epoch 8/100
25s - loss: 0.1417 - acc: 0.9523 - val_loss: 0.1265 - val_acc: 0.9542
Epoch 9/100
25s - loss: 0.1687 - acc: 0.9415 - val_loss: 0.1982 - val_acc: 0.9150
Epoch 10/100
25s - loss: 0.1306 - acc: 0.9585 - val_loss: 0.1047 - val_acc: 0.9564
Epoch 11/100
25s - loss: 0.1419 - acc: 0.9474 - val_loss: 0.1131 - val_acc: 0.9542
Epoch 12/100
25s - loss: 0.1327 - acc: 0.9569 - val_loss: 0.1016 - val_acc: 0.9630
Epoch 13/100


In [25]:
test_images = test_images.astype('float32')
test_images /= 255

In [27]:
preds = []
for i, model in enumerate(models):
    model.load_weights(w_file.format(i))
    pred = model.predict(test_images)
    preds.append(pred)
preds = np.mean(preds, axis=0)

In [29]:
sample_submission = pd.read_csv(DATA_DIR+"/sample_submission.csv")

for i, name in enumerate(test_names):
    sample_submission.loc[sample_submission['name'] == name, 'invasive'] = preds[i]

sample_submission.to_csv("submit2.csv", index=False)